In [1]:
from fileinput import filename
import numpy as np
from copy import deepcopy
from tkinter import ttk

import random
import csv
from PIL import Image
from PIL import Image as im
from numpy import asarray
import cv2
import scipy

import math
import tkinter as tk

from PIL import Image, ImageTk, ExifTags
from tkinter import filedialog
from tkinter.filedialog import askopenfile
from matplotlib import pyplot as plt

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)

In [2]:
root = tk.Tk()
root.title('Decryption')

''

In [3]:
#pip install --upgrade tables

In [4]:
def binaryToDecimal(binary):
 
    decimal, i = 0, 0
    while(binary != 0):
        dec = binary % 10
        decimal = decimal + dec * pow(2, i)
        binary = binary//10
        i += 1
    return decimal

In [5]:
def pol_val(cur_i,subi, N ,x_coord,y_coord,position_values):
    
#     coefficient =np.poly1d( generate_polynomial((t-1)-ni , prime))
#   randVal=(np.polyval(coefficient,subi))%prime
#    tempi=1
#     for i in range(1,ni+1):
#             tempi=(tempi*(subi-i))%prime
#     tempi=(tempi*randVal)%prime
    
    tempix=0
    
    for i in range(0,N):
        tempj=1
        i_val = x_coord[cur_i][i]
        
        for j in range(0,N):
            j_val = y_coord[cur_i][j]
            if i_val!=j_val:
                subi_i=subi-j_val
                i_j=i_val-j_val
                mul=subi_i/i_j
#                 print(subi_i)
#                 print(i_j)
                tempj=(tempj*mul)%257
#                 print(i," ",j," ",subi," ",abs(subi_i)," ",i_j)
        tempix=(tempix+(position_values[cur_i][i]*tempj))%257
    qxi=(tempix)%257
    
    
#     print(qxi,qxj)
    return round(qxi)

In [6]:
def collect_images(N):


    shadow_images = []
    for i in range(0,N+1):
        image = Image.open("ShadowImage"+str(i)+".png")
        numpydata = np.asarray(image)
        shadow_images.append(deepcopy(numpydata))
        print("For ",i," ",numpydata.shape)
    
    return shadow_images

In [7]:
def check_writable(img):
    for i in img:
        print("flag value")
        i.setflags(write=1)
        print(i.flags)

In [8]:
def calculate_positions(m1,m2,N,shadow_images):
    position_values = []
    x_coord = []
    y_coord = []

    for i in range(0,m1): 
        for k in range(0,m2):
            pos_i = []
            x = []
            y = []
            for j in range(1,N+1):
                pos_i.append(int(shadow_images[j][i][k])^int(shadow_images[0][i][k]))
                x.append(j)
                y.append(j)

            position_values.append(pos_i)
            x_coord.append(x)
            y_coord.append(y)
    return position_values , x_coord,y_coord

In [9]:
# def data_embed(m1,m2,N,shadow_images,text):
#     secret_text =""
#     str_sectxt = ""
#     print('writing the text')
#     #check_writable(shadow_images)
    
#     for i in range(0,len(text)):
#         img_number = (i%N )+1
#         position = i%(m1*m2)
#         bit_position = i % 8
#         position_x = position // m2
#         position_y = position % m2
#         numpydata=[]
        
# #         image = Image.open('ShadowImage'+str(img_number)+'.png')
# #         numpydata = np.asarray(image)
# #         numpydata=numpydata.astype(int)
        
#         numpydata = list(shadow_images[img_number])
        
#         positionval = format(numpydata[position_x][position_y],'08b')
#         print("for ith bit", i, " " , positionval)
#         var_int = int(text[i])
#         position_int = int(positionval[bit_position])
#         original_bit = var_int ^ position_int
        
#         secret_text = secret_text + positionval[bit_position] 
#         if(len(secret_text) == 8):
#             #print((secret_text))
#             str_sectxt = str_sectxt + chr(binaryToDecimal(int(secret_text)))
#             secret_text = ""
            
            
        
#         pos_var = (positionval[0:bit_position])+ str(original_bit)
       
        
#         if(bit_position != 7):
#             pos_var = str(pos_var) + positionval[bit_position+1:]
#         #positionval[-bit_position] = var[i]
#         numpydata[position_x][position_y] = binaryToDecimal(int(pos_var))
# #        shadow_images[img_number][position_x][position_y] = int(pos_var)
#         shadow_images[img_number] = numpydata
    
#         print("for i ",i,"ith bit " , img_number , "pos x" ,position_x , "pos y", position_y ,"bit pos",bit_position )
#         print(numpydata[position_x][position_y])
    
#     print(secret_text)
#     print(str_sectxt)
#     return str_sectxt,shadow_images

In [10]:
def openMetricsTable(menu):
    
    if(menu == 'salt and pepper filter'):
        values = ((31.48983005,33.53742834,30.08826859,31.85994615,31.49414189),
                  (0.8637,0.8107111111,0.898425,0.8222156379,0.839),(46.1421,28.79635556,63.71673333,42.37266173,46.09631111))
    elif (menu == 'poission noise filter'):
        values = ((29.00952934,29.0864927,28.73386926,28.85524403,28.79399422),
                  (0.974,0.9734666667,0.9783416667,0.9769547325,0.9784444444),(81.6822,80.24742222,87.0349,84.63616132,85.83826667))
    elif(menu =='rotation filter'): 
        values = ((Infinity,Infinity,Infinity,Infinity,Infinity),
                  (0.0,0.0,0.0,0.0,0.0),(0,0,0,0,0))
        
    newWindow = tk.Toplevel(root)
    newWindow.title("table")
    newWindow.geometry("1000x500") 
    
    tk.Label(newWindow,text =menu.upper() + " METRICS Values").pack()
    s = ttk.Style()
    s.theme_use('clam')
    s.configure('Treeview',rowheight = 50,font=('Helvetica',16))

    #table = tk.Frame(newWindow)

    t=ttk.Treeview(newWindow,height=7)
    t['columns']= ('Images','PSNR','BER','MSE')

    t.column("#0", width=0,  stretch=tk.NO)
    t.column("Images",anchor=tk.CENTER, width=300)
    t.column("PSNR",anchor=tk.CENTER, width=200)
    t.column("BER",anchor=tk.CENTER,width=200)
    t.column("MSE",anchor=tk.CENTER,width=200)

    t.heading("#0",text="",anchor=tk.CENTER)
    t.heading("Images",text="images",anchor=tk.CENTER)
    t.heading("PSNR",text="psnr value",anchor=tk.CENTER)
    t.heading("BER",text="ber value",anchor=tk.CENTER)
    t.heading("MSE",text="mse value",anchor=tk.CENTER)



    t.insert(parent = '' , index='end',iid=0,text='',values=('100*100 - flowers ',values[0][0],values[1][0],values[2][0]))
    t.insert(parent = '' , index='end',iid=1,text='',values=('150*150 - parrots ',values[0][1],values[1][1],values[2][1]))
    t.insert(parent = '' , index='end',iid=2,text='',values=('200*200 - rays[RGB]',values[0][2],values[1][2],values[2][2]))
    t.insert(parent = '' , index='end',iid=3,text='',values=('256*256 - lena [RGB]',values[0][3],values[1][3],values[2][3]))
    t.insert(parent = '' , index='end',iid=4,text='',values=('300*300 - snail',values[0][4],values[1][4],values[2][4]))



#     canvas = FigureCanvasTkAgg(fig, master=root)
#     canvas.draw()
#     canvas.get_tk_widget().pack()
    #table.pack()
    t.pack()

In [11]:
def openOriginalImage(img2,menu,N):
    
    img2 = cv2.imread('DecryptedImage.png')
    median =[]
    data = []
    
    
    if(menu=='salt and pepper filter'):
        median = cv2.medianBlur(np.array(img2),5)
        
        b,g,r = cv2.split(median)
        for i in range(0,median.shape[0]):
            for j in range(0,median.shape[1]):
                median[i][j] = [r[i][j] , g[i][j], b[i][j]]
    
        data=im.fromarray(np.array(median).astype(np.uint8))
        data.save('DecryptedImage.png')
        
        
    elif(menu == 'poission noise filter'):
#         median = cv2.fastNlMeansDenoising(img2, None, 27, 21, 7)

        median = cv2.medianBlur(np.array(img2),3)
        
        b,g,r = cv2.split(median)
        for i in range(0,median.shape[0]):
            for j in range(0,median.shape[1]):
                median[i][j] = [r[i][j] , g[i][j], b[i][j]]
    
        data=im.fromarray(np.array(median).astype(np.uint8))
        data.save('DecryptedImage.png')
    
#         data=im.fromarray(np.array(median).astype(np.uint8))
#         data.save('DecryptedImage.png')
        
#         data=cv2.GaussianBlur(np.array(img2),(3,3),0)
#         data.save('DecryptedImage.png')
        #median = np.asarray(median)
    
    
    
#         data=scipy.signal.wiener(img2,(3,3))
#         data.save('DecryptedImage.png')
        
    elif(menu == 'rotation filter'):

        
        for i in range(1,N+1):
            
            image = Image.open("ShadowImage"+str(i)+".png")
            numpydata = np.asarray(image)
            numpydata = cv2.rotate(numpydata, cv2.ROTATE_180)
            #numpydata = cv2.fastNlMeansDenoising(numpydata, None, 5, 21, 7)
            
            print("For ",i," ",numpydata.shape)
        
        
            data=im.fromarray(np.array(numpydata).astype(np.uint8))
            data.save('ShadowImage'+str(i)+'.png')
            
        decrypt()   
        

    
    #data=im.fromarray(np.array(median).astype(np.uint8))
    #data.save('DecryptedImage.png')
       
    img2 = Image.open('DecryptedImage.png')
    img2 = img2.resize((300, 300))
    img2 = ImageTk.PhotoImage(img2)
    img_label2 = tk.Label(image=img2)
    img_label2.image = img2
    img_label2.grid(column=1, row=16)
    
    window_btn = tk.Button(root,text ="View Metrics Table",command = lambda: openMetricsTable(menu),font=("Raleway", 13), bg="#000000", fg="white", height=2, width=30)
    window_btn.grid(column =1,row=18)

In [12]:
def decrypt():
    
    
    N=0
    m1=0
    m2=0
    b=0
    
    with open("Nandm1andm2.txt","r") as f:
        f.readline()
        attr=f.readline().split(",")
        N=int(attr[0])
        m1=int(attr[1])
        m2=int(attr[2])
        b=int(attr[3])
        text = attr[4]
    
    shadow_images = collect_images(N)
    
        
#         text = text + str((position_int)^(var_int))
        
        
#         data=im.fromarray(np.array(numpydata).astype(np.uint8))
#         data.save('ShadowImage'+str(img_number)+'.png')
#         print("for i ",i,"ith bit " , img_number , "pos x" ,position_x , "pos y", position_y ,"bit pos",bit_position )
    
    
      

    Original_image=[]
    if(b>0):
        ori_array_r=[]
        ori_array_g=[]
        ori_array_b=[]

        shad_r=[]
        shad_g=[]
        shad_b=[]
        for i in range(0,N+1):
            shad_r.append(0)
            shad_g.append(0)
            shad_b.append(0)
            shad_b[i],shad_g[i],shad_r[i]=cv2.split(shadow_images[i])
#         for i in range(1,N+1):
#             ori_array_r=[*ori_array_r,*(shad_r[i]^shad_r[i-1])]
#             ori_array_g=[*ori_array_g,*(shad_g[i]^shad_g[i-1])]
#             ori_array_b=[*ori_array_b,*(shad_b[i]^shad_b[i-1])]
    #     print(len(ori_array_r))
        
        #str_sectxt =''
        
        #str_sectxt ,shad_b = data_embed(m1,m2,N,shad_b,text)
        print("shadow imag r " , list(shad_r))
        
        position_valuesR,x_coordR,y_coordR = calculate_positions(m1,m2,N,shad_r)
        
        position_valuesG,x_coordG,y_coordG = calculate_positions(m1,m2,N,shad_g)
        position_valuesB,x_coordB,y_coordB = calculate_positions(m1,m2,N,shad_b)
        
        for i in range(1,N+1):
            col = 0
            Img = []

            #print(">>>>For ",i,"th n value<<<<")
            for j in range(0,m1):
                I_row =[]
                for k in range(0,m2):
                    qxr = pol_val(((j*m2)+k),i,N , x_coordR,y_coordR,position_valuesR)
                    qxg = pol_val(((j*m2)+k),i,N , x_coordG,y_coordG,position_valuesG)
                    qxb = pol_val(((j*m2)+k),i,N , x_coordB,y_coordB,position_valuesB)

                    I_row.append([qxr,qxg,qxb])
                Img.append(I_row)

            Original_image=[*Original_image,*Img]

        data=im.fromarray(np.array(Original_image).astype(np.uint8))
        data.save('DecryptedImage.png')
        
    else :
        #str_sectxt ,shadow_images = data_embed(m1,m2,N,shadow_images,text)
        position_values,x_coord,y_coord = calculate_positions(m1,m2,N,shadow_images)
        
        
        for i in range(1,N+1):
            col = 0
            Img = []

            #print(">>>>For ",i,"th n value<<<<")
            for j in range(0,m1):
                I_row =[]
                for k in range(0,m2):
                    qxi = pol_val(((j*m2)+k),i,N , x_coord,y_coord,position_values)

                    I_row.append(round(qxi))
                Img.append(I_row)

            Original_image=[*Original_image,*Img]
        
        data=im.fromarray(np.array(Original_image).astype(np.uint8))
        data.save('DecryptedImage.png')
      
        
    img2 = Image.open('DecryptedImage.png')
    img2 = img2.resize((300, 300))
    img2 = ImageTk.PhotoImage(img2)
    img_label2 = tk.Label(image=img2)
    img_label2.image = img2
    img_label2.grid(column=1, row=2)
    
#     tk.Label(root,text="THE SECRET TEXT IS:",font = ('Helvetica', 18, 'bold')).grid(column= 1 ,row=6,  ipadx = 15 , ipady  = 5)
#     inputtxt = tk.Label(root,text=str_sectxt,font = ('Helvetica', 15))
# #     inputtxt.insert(0,str_sectxt)
#     inputtxt.grid(column = 2 , row = 6 , ipadx = 15 , ipady = 5 , padx= 0)
    
#     print(position_values)
#     print(x_coord)
#     print(y_coord)
    
    menu= tk.StringVar()
    menu.set("Select Any one of the recovery filter")
    drop= tk.OptionMenu(root, menu,"salt and pepper filter", "poission noise filter","rotation filter")
    drop.grid(column=1,row=15)
    
    window_btn = tk.Button(root,text ="View Filtered Original image",command = lambda: openOriginalImage(img2,menu.get(),N),font=("Raleway", 13), bg="#000000", fg="white", height=2, width=30)
    window_btn.grid(column =1,row=16)
    
    

In [13]:
button_text = tk.StringVar()
send_btn_text = tk.StringVar()
send_btn = tk.Button(root, textvariable=send_btn_text, command=lambda: decrypt(),
                     font=("Raleway", 13), bg="#000000", fg="white", height=2, width=12)
send_btn_text.set("decrypt")
send_btn.grid(column=1, row=0)

In [14]:
canvas = tk.Canvas(root, width=600, height=300)
canvas.grid(columnspan=3, rowspan=3)

root.mainloop()

For  0   (15, 150)
For  1   (15, 150)
For  2   (15, 150)
For  3   (15, 150)
For  4   (15, 150)
For  5   (15, 150)
For  6   (15, 150)
For  7   (15, 150)
For  8   (15, 150)
For  9   (15, 150)
For  10   (15, 150)
